# MCNN的特征图和卷积核的可视化


## 加载模型
MCNN的test文件里面有加载模型的代码,直接使用

In [1]:
import os
import torch
import numpy as np
import sys
import torch.nn as nn

from src.crowd_count import CrowdCounter
from src import network
from src.data_loader import ImageDataLoader
from src.timer import Timer
from src import utils
from src.evaluate_model import evaluate_model

torch.backends.cudnn.enabled = True
torch.backends.cudnn.benchmark = False
vis = False
save_output = True

#使用mcnn的test文件的代码加载mcnn模型


data_path =  './data/original/shanghaitech/part_A_final/test_data/images/'
gt_path = './data/original/shanghaitech/part_A_final/test_data/ground_truth_csv/'
model_path = './saved_models/mcnn_shtechA_660.h5'



output_dir = './output/'
model_name = os.path.basename(model_path).split('.')[0]
file_results = os.path.join(output_dir,'results_' + model_name + '_.txt')
if not os.path.exists(output_dir):
    os.mkdir(output_dir)
output_dir = os.path.join(output_dir, 'density_maps_' + model_name)
if not os.path.exists(output_dir):
    os.mkdir(output_dir)


net = CrowdCounter()
      
trained_model = os.path.join(model_path)
network.load_net(trained_model, net)
net.cuda()
net.eval()
mae = 0.0
mse = 0.0

加载的模型存在net中,是CrowdCounter()的类型
![](https://raw.githubusercontent.com/iicanfly/screenshot/v-mcnn/1.png)
从上图我们看到net.DME存的是MCNN(),因此将net.DME打印出来
![](https://raw.githubusercontent.com/iicanfly/screenshot/v-mcnn/2.png)
以Conv2d(1, 16, kernel_size=(9, 9), stride=(1, 1), padding=(4, 4))为例,输入通道为1,输出通道为16,卷积核个数与输出通道相等即为16,卷积核大小为9*9,池化大小为4*4

In [2]:
net.DME #打印出mcnn神经网络的结构

MCNN(
  (branch1): Sequential(
    (0): Conv2d(
      (conv): Conv2d(1, 16, kernel_size=(9, 9), stride=(1, 1), padding=(4, 4))
      (relu): ReLU(inplace)
    )
    (1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (2): Conv2d(
      (conv): Conv2d(16, 32, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3))
      (relu): ReLU(inplace)
    )
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(
      (conv): Conv2d(32, 16, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3))
      (relu): ReLU(inplace)
    )
    (5): Conv2d(
      (conv): Conv2d(16, 8, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3))
      (relu): ReLU(inplace)
    )
  )
  (branch2): Sequential(
    (0): Conv2d(
      (conv): Conv2d(1, 20, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3))
      (relu): ReLU(inplace)
    )
    (1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (2): Conv2d(
      (conv): Conv2

由net的load_net文件我们可以知道模型的参数存在state_dict()中,先打印出来看看
![](https://raw.githubusercontent.com/iicanfly/screenshot/v-mcnn/3.png)

In [3]:
params=net.state_dict()   #将net的信息存入params中
for k,v in params.items():
    print(k)    #打印网络中的变量名

DME.branch1.0.conv.weight
DME.branch1.0.conv.bias
DME.branch1.2.conv.weight
DME.branch1.2.conv.bias
DME.branch1.4.conv.weight
DME.branch1.4.conv.bias
DME.branch1.5.conv.weight
DME.branch1.5.conv.bias
DME.branch2.0.conv.weight
DME.branch2.0.conv.bias
DME.branch2.2.conv.weight
DME.branch2.2.conv.bias
DME.branch2.4.conv.weight
DME.branch2.4.conv.bias
DME.branch2.5.conv.weight
DME.branch2.5.conv.bias
DME.branch3.0.conv.weight
DME.branch3.0.conv.bias
DME.branch3.2.conv.weight
DME.branch3.2.conv.bias
DME.branch3.4.conv.weight
DME.branch3.4.conv.bias
DME.branch3.5.conv.weight
DME.branch3.5.conv.bias
DME.fuse.0.conv.weight
DME.fuse.0.conv.bias


In [4]:
bia = params['DME.branch1.0.conv.bias']     #将branch1.0中的偏置参数存入bia中
weigh = params['DME.branch1.0.conv.weight']     #将branch1.0中的卷积核存入weigh中

In [5]:
WW = weigh.cpu().numpy();    #将torch的tensor转化为numpy的数组类型
BB = bia.cpu().numpy()

将DME.branch1.0.conv的卷积核信息输出,有16个单通道卷积核,大小为9*9

In [6]:
print(WW.shape)  
print(WW)   

(16, 1, 9, 9)
[[[[-1.79500356e-02 -1.99243557e-02 -3.84157221e-03 ...  6.16973033e-04
    -7.51812570e-03  7.86256790e-03]
   [ 2.25035567e-02 -6.46710489e-03  3.01630585e-03 ... -2.51524150e-02
    -1.54171605e-02 -1.53352702e-02]
   [ 5.84645988e-03  2.51216791e-03 -1.13419117e-02 ... -3.46027426e-02
    -1.73162576e-02 -5.64664938e-02]
   ...
   [ 1.74859352e-02  2.07951944e-02  1.28669590e-02 ... -1.26834009e-02
    -9.77535639e-03 -7.61114201e-03]
   [ 2.64665391e-02  2.69219074e-02  2.23359372e-02 ... -8.00045207e-03
     1.00213140e-02 -1.52185000e-02]
   [ 7.54476245e-03  1.25497393e-02 -5.67897037e-03 ... -1.18124578e-02
     1.63770874e-03  3.32620274e-03]]]


 [[[-4.09158785e-03 -7.06498092e-03  1.33825950e-02 ...  9.12127551e-03
     3.44570354e-02  7.98309967e-03]
   [ 1.49876466e-02  3.60761816e-03 -2.69855503e-02 ... -3.69569496e-03
     1.94978174e-02 -2.11405996e-02]
   [ 5.67956631e-05  9.48098581e-03  2.70815808e-02 ... -3.06394305e-02
     1.24712083e-02  1.94623265

## 卷积核的可视化(比较粗浅的实现方式,有待改进)
将卷积核的元素归一化到0-1之间,然后再相应放大到0-255之间

In [7]:
import cv2
WW[0,0] -= WW[0,0].min()
WW[0,0] /= WW[0,0].max()
WW[0,0] = WW[0,0] * 255
for i in range(16):      #for循环可视化16个卷积核
    WW[i,0] -= WW[i,0].min()
    WW[i,0] /= WW[i,0].max()
    WW[i,0] = WW[i,0] * 255
    cv2.imwrite('./Visualization generated/{}.png'.format(i),WW[i,0], [int(cv2.IMWRITE_PNG_COMPRESSION), 0])

结果可以得到16张类似下图的可视化后的卷积核
![](https://raw.githubusercontent.com/iicanfly/screenshot/v-mcnn/5.png)

## 特征图的可视化
思路:
1. 使用一张MCNN项目中的图片
2. 确定好想要可视化第几层的特征图,以及是哪一个卷积核卷积后的特征图
3. 使用conv2d定义中的forward()方法将图片往前传播,由于上面分析我们知道卷积核的矩阵为[X,1,D,D],其中X为卷积核个数,1代表单通道,D代表卷积核大小
4. 由于灰度图为二维,因此图片不能直接传入,需要经过preprocess_image()方法:使用mean和std对0-255的二维numpy数组进行处理,然后转化为float型的torch变量,然后使用unsqueeze_(0)方法将二维数组变成四维数组,才能传进forward中
5. 为了凸显出改特征图的特征,将第一次卷积后得到的特征图重新输入卷积神经网络,重复多次,可以看到特征越来越明显
6. 重新输入卷积神经网络的时候要用mean和std对数据进行与第4点相反的处理,再输入

In [8]:
import os
import numpy as np

import torch
from torch.optim import Adam
from torchvision import models

from misc_functions import preprocess_image, recreate_image, save_image


class CNNLayerVisualization():
    
    def __init__(self, model, selected_layer, selected_filter,imgg):
        self.model = model
        self.model.eval()
        self.selected_layer = selected_layer
        self.selected_filter = selected_filter
        self.conv_output = 0
        self.created_image = imgg
      
        # 创建文件夹
        if not os.path.exists('./Visualization generated'):
            os.makedirs('./Visualization generated')


    def visualise_layer_without_hooks(self):
        # 将图像进行处理,转化为4维float tensor
        self.processed_image = preprocess_image(self.created_image)
        #定义一个优化器
        optimizer = Adam([self.processed_image], lr=0.1, weight_decay=1e-6)
        #重复输入31次MCNN网络
        for i in range(1, 31):
            optimizer.zero_grad()
            # x作为向前传播的图像
            x = self.processed_image
            x = x.cpu()
            x = x.cuda()
            #x = x.double()
            x = self.model.DME.branch1[0].forward(x)            
            #选择特定的卷积核卷积的特征图
            self.conv_output = x[0, self.selected_filter]
            #loss函数使用特征图的平均值
            loss = -torch.mean(self.conv_output)
            print('Iteration:', str(i), 'Loss:', "{0:.2f}".format(loss.data.cpu().numpy()))
            # loss对图像像素求导
            loss.backward()
            # 更新图像像素
            optimizer.step()
            # 重新处理图像,以重新输入MCNN网络
            self.created_image = recreate_image(self.processed_image)
            # 保存图像
            if i % 5 == 0:
                im_path = './Visualization generated/layer_vis_l' + str(self.selected_layer) + \
                    '_f' + str(self.selected_filter) + '_iter' + str(i) + '.jpg'
                save_image(self.created_image, im_path)
  


    def save_image(im, path):
   
        if isinstance(im, np.ndarray):
            if len(im.shape) == 2:
                im = np.expand_dims(im, axis=0)
            if im.shape[0] == 1:
                #此时传进来的图像为3维的,其中第一维长度为1,可以看成图像的通道数,使用repeat拓展为3个通道,且每个通道的数据一样
                im = np.repeat(im, 3, axis=0)
                #经过上面处理,第一维为通道数,先对数组使用transpose进行倒转,再存储
            if im.shape[0] == 3:
                im = im.transpose(1, 2, 0) * 255
            im = Image.fromarray(im.astype(np.uint8))
        im.save(path)


    def preprocess_image(pil_im, resize_im=False):

        mean = 0.485
        std = 0.229
        #使用mean和std对图像进行处理
        im_as_arr = np.float32(pil_im)
        im_as_arr /= 255
        im_as_arr -= mean 
        im_as_arr /= std
        # 转化为float tensor类型
        im_as_ten = torch.from_numpy(im_as_arr).float()
        # 由于卷积核为四维,因此需要拓展成4维的tensor
        im_as_ten.unsqueeze_(0)
        im_as_ten.unsqueeze_(0)
        # 转为Variable变量,设置可进行求导操作
        im_as_var = Variable(im_as_ten, requires_grad=True)
        return im_as_var


    def recreate_image(im_as_var):

        reverse_mean = -0.485
        reverse_std = 1/0.229
        #此时图像由4维变成3维
        recreated_im = copy.copy(im_as_var.data.numpy()[0])
        recreated_im /= reverse_std
        recreated_im -= reverse_mean
     #   recreated_im[recreated_im > 1] = 1
     #   recreated_im[recreated_im < 0] = 0
        recreated_im = np.round(recreated_im * 255)
        return recreated_im

    
if __name__ == '__main__':
    #方便起见,可视化第一层的第一个卷积核卷积得到的特征图
    #若想可视化后面层数的特征图,加入for循环使用farward()得到
    cnn_layer = 1
    filter_pos = 1
    pretrained_model = net
    imgg = cv2.imread("2_5.jpg",0)
    layer_vis = CNNLayerVisualization(pretrained_model, cnn_layer, filter_pos,imgg)

    layer_vis.visualise_layer_without_hooks()

('Iteration:', '1', 'Loss:', '-0.04')
('Iteration:', '2', 'Loss:', '-0.04')
('Iteration:', '3', 'Loss:', '-0.04')
('Iteration:', '4', 'Loss:', '-0.05')
('Iteration:', '5', 'Loss:', '-0.06')
('Iteration:', '6', 'Loss:', '-0.07')
('Iteration:', '7', 'Loss:', '-0.07')
('Iteration:', '8', 'Loss:', '-0.08')
('Iteration:', '9', 'Loss:', '-0.09')
('Iteration:', '10', 'Loss:', '-0.10')
('Iteration:', '11', 'Loss:', '-0.11')
('Iteration:', '12', 'Loss:', '-0.12')
('Iteration:', '13', 'Loss:', '-0.13')
('Iteration:', '14', 'Loss:', '-0.14')
('Iteration:', '15', 'Loss:', '-0.16')
('Iteration:', '16', 'Loss:', '-0.17')
('Iteration:', '17', 'Loss:', '-0.18')
('Iteration:', '18', 'Loss:', '-0.19')
('Iteration:', '19', 'Loss:', '-0.20')
('Iteration:', '20', 'Loss:', '-0.21')
('Iteration:', '21', 'Loss:', '-0.22')
('Iteration:', '22', 'Loss:', '-0.24')
('Iteration:', '23', 'Loss:', '-0.25')
('Iteration:', '24', 'Loss:', '-0.26')
('Iteration:', '25', 'Loss:', '-0.27')
('Iteration:', '26', 'Loss:', '-0.

原图像为
![](https://raw.githubusercontent.com/iicanfly/screenshot/v-mcnn2/2_5.jpg)

将第5次输入,第10次.......第30次输入得到特征图列出来看一看
![](https://raw.githubusercontent.com/iicanfly/screenshot/v-mcnn2/layer_vis_l1_f1_iter5.jpg)
![](https://raw.githubusercontent.com/iicanfly/screenshot/v-mcnn2/layer_vis_l1_f1_iter10.jpg)
![](https://raw.githubusercontent.com/iicanfly/screenshot/v-mcnn2/layer_vis_l1_f1_iter15.jpg)
![](https://raw.githubusercontent.com/iicanfly/screenshot/v-mcnn2/layer_vis_l1_f1_iter20.jpg)
![](https://raw.githubusercontent.com/iicanfly/screenshot/v-mcnn2/layer_vis_l1_f1_iter25.jpg)
![](https://raw.githubusercontent.com/iicanfly/screenshot/v-mcnn2/layer_vis_l1_f1_iter30.jpg)